<a href="https://colab.research.google.com/github/LoboViejo61/LoboViejo.github.io/blob/main/Scrapeando_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Cambia la keyword por la que quieras y dale al PLAY (ej. keyword = "mejores cafeteras 2022")
keyword = "Belleza y salud"
encabezados = ['h1', 'h2', 'h3']

# solicitud HTTP
import requests
# LLamo a la libreria BeautifulSoup
from bs4 import BeautifulSoup

# re expresion regulares
import re

# herramienta de lectura de lenguaje natural
import nltk

class pagina():
    # estructura las url
    def __init__(self, url):
        self.url = url
        self.texto = ""
        self.enlaces  = []
        self.headings = []

class serp():
    # init method or constructor /metodo contructor
    def __init__(self, query):
        self.query = query.replace(" ", "+")
        self.ok = False
        self.incidendias = []
        self.paginas = []
        self.start()
# contructor de la busqueda de google
    def start(self):
        # generar una busquedad en google basado en nuentro requerimiento
        URL = "https://www.google.com/search?hl=en&gl=uk&q=%s&oq=%s" % (self.query, self.query)
        # genero un identificador de navegador y sistema operativo para evadir las validaciones de google
        headers =  {"user-agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.104 Safari/537.36"}
        # variable donde va contener resultados de Serp
        enlacesSerp = []
        # variable contenedora de la url mas la cabecera de indentificacion
        resp = requests.get(URL, headers = headers)
        # condicion que debe ser o validar que la respuesta del servidor = 200
        if resp.status_code == 200:
            # declaro un objeto que se construye basado en el resultado de la libreria BeautifulSoup que contiene todos los elementos HTML de una url
            soup = BeautifulSoup(resp.content, "html.parser")
            # creo el objeto link para hacer una busqueda dentro de los contenedores
            # referenciados con la etiqueta DIV los cuales internamente tiene las class
            links = soup.find_all("div", {"class" : "g"})
            # visualice en pantalla las url encontradas
            print("Procesando urls:")

            # bucle de procesos con el for
            for x in links:
                # genero un variable
                links0 = x.find_all("a", href=True)
                # len cuenta la cantidad de caracteres para este caso
                if len(links0) == 0:continue
                link = links0[0]['href']
                #corrijo futuros snippet (termino que se utiliza para la comprobacion de pequeños codigos)
                if "#:~:text" in link:
                    link = link.split("#:~:text")[0]
                if link.startswith("/"):continue
                print(link)
                # que es para validar si algo no exite
                if not link in enlacesSerp:
                    enlacesSerp.append(link)

                    try:
                      resp = requests.get(link, headers = headers, timeout=10)

                    except Exception as e:
                      self.incidendias.append(f"timeout error: {str(e)} url: {link}")

                    soup = BeautifulSoup(resp.content, "html.parser")

                    if resp.status_code == 200: # valido si el codigo 200 de buena conexion y carga
                        pag = pagina(link)

                        #elimino scrips
                        for s in soup.find_all(['script', 'style']):
                            s.extract()

                        #recupero texto
                        texto = ""
                        tag = soup.body
                        for string in tag.strings:
                            texto = texto + " " + string
                        pag.texto = texto

                        #recupero enlaces
                        for en in soup.find_all("a", href=True):
                            if "#" in en["href"] or "http" in en["href"]:continue
                            if en.text not in pag.enlaces:
                                pag.enlaces.append(en.text)

                        #recupero headings
                        for heading in soup.find_all(encabezados):
                            if heading.text not in pag.headings:
                                pag.headings.append([heading.name, heading.text.strip('\n').strip()])

                        self.paginas.append(pag)
                    else:
                        self.incidendias.append(f"Status code: {resp.status_code} url: {link}")

            if len(self.paginas) < 5:
                self.ok = True
                self.incidendias.append("Menos de 5 resultados escrapeados")
            else:
                self.ok = True

        else:
            self.ok = False

        #reporto incidencias scrapeo
        print(str(len(self.paginas)) + " páginas escrapeadas correctamente")

        if len(self.incidendias) > 0:
            print("Incidencias")
            print("-----------")
            for x in self.incidendias:
                print(x)

scrap = serp(keyword)

if scrap.ok == False:
    print("No se ha completado el scrap, se detiene el análisis => Sorry...")
    quit()
# --------------
#Creo el modelo word2Vec => paso de palabras a vectores
print("Instalando dependencias...")
!pip install nltk
!pip install gensim
from gensim.models import Word2Vec
from unicodedata import normalize
import spacy
nltk.download('stopwords')
nltk.download('punkt')
!python -m spacy download es_core_news_sm
import es_core_news_sm
nlp = es_core_news_sm.load()

print("Pasando texto scrapeado a vectores y creando modelo word2Vec...")
texto = ""
for p in scrap.paginas:
    texto = texto + " " + p.texto
texto = texto.strip()

def lematizarFrase(frase):
    doc = nlp(frase)
    lematizado = [token.lemma_ for token in doc]
    return " ".join(lematizado)

def quitarAcentos(frase):
  frase = re.sub(
        r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1",
        normalize( "NFD", frase), 0, re.I
    )
  frase = normalize( 'NFC', frase)
  return frase

from nltk.corpus import stopwords
def quitarStopWords(frase):
  tokenizado = frase.split()
  tokenizado = [w for w in tokenizado if w not in stopwords.words('spanish')]
  return " ".join(tokenizado)

def limpiarBasico(frase, lemat = True):
    frase = re.sub('[^a-zA-ZÀ-ÿ\u00f1\u00d1]', ' ', frase)
    frase = re.sub(r'\s+', ' ', frase)
    frase = frase.replace("  ", " ").strip()
    if lemat == True: frase = lematizarFrase(frase)
    frase = quitarAcentos(frase)
    return frase

def limpia(texto):
    procesando = limpiarBasico(texto)
    procesando = procesando.lower()
    procesando = procesando.replace("(", "").replace(")", "")

    frases = nltk.sent_tokenize(procesando)
    tokenizado = [nltk.word_tokenize(sent) for sent in frases]

    # Removing Stop Words
    for i in range(len(tokenizado)):
        tokenizado[i] = [w for w in tokenizado[i] if w not in stopwords.words('spanish')]
    return tokenizado

textoLimpio = limpia(texto)
from gensim.models import Word2Vec

word2vec = Word2Vec(textoLimpio, min_count=2, epochs=2)
modelo = word2vec.wv
print("Modelo creado => Puntuando headings/enlaces...")

vocabulario = modelo.index_to_key

def limpiarFrase(frase):
    frase = frase.lower()
    frase = frase.replace("(", "").replace(")", "")
    frase = limpiarBasico(frase)
    frase = frase.split()
    frase = [x for x in frase if x in vocabulario]
    return " ".join(frase).strip()

headings = []
for pagina in scrap.paginas:
    for heading in pagina.headings:
      he = []

      he.append(heading[1].strip())
      sinStop = quitarStopWords(heading[1])
      he.append(limpiarFrase(sinStop))
      headings.append(he)

enlaces = []
for pagina in scrap.paginas:
    for enlace in pagina.enlaces:
      he = []
      he.append(enlace.strip())
      sinStop = quitarStopWords(enlace)
      he.append(limpiarFrase(sinStop))
      enlaces.append(he)


headings = [hea for hea in headings if hea[1].strip() !=""]
enlaces = [hea for hea in enlaces if hea[1].strip() !=""]

keys = [x for x in limpiarBasico(keyword).lower().split() if x in vocabulario]

#puntuo los enlaces/headings por relevancia con la keyword
def puntuar(keyword, lista):
  for en in lista:
    palabrasRelev = [palabra for palabra in en[1].split()]
    res = round(modelo.n_similarity(keys, palabrasRelev),3)
    en.append(str(res))
  lista = sorted(lista, key=lambda x: x[2], reverse=True)
  return lista


headings = puntuar(keyword, headings)
enlaces = puntuar(keyword,enlaces)


def agrupar(lista0):
  lista = lista0.copy()
  headingsAgrupados = []
  while len(lista)>0:
    relacionados = []
    primero = lista.pop(0)
    palabrasPrimero = [palabra for palabra in primero[1].split()]

    for hea in list(lista):
      palabrasRelev = [palabra for palabra in hea[1].split()]
      res = round(modelo.n_similarity(palabrasPrimero, palabrasRelev),3)

      if res == 1:
        if hea[0] not in relacionados and hea[0] != primero[0]:
          relacionados.append(hea[0])
        lista.remove(hea)
    relacionados = " - ".join(relacionados)
    primero.append("")
    primero.append(relacionados)
    headingsAgrupados.append(primero)
  return headingsAgrupados

#Agrupo los headings similares
headingsFinal = agrupar(headings)

#borro duplicados enlaces
enlaces2 = []
for enlace in enlaces:
  anchors = [x[0].lower() for x in enlaces2]
  if enlace[0].lower() not in anchors:
    enlaces2.append(enlace)
enlaces = enlaces2

#Paso de listas a tablas
import pandas as pd

headingsFinal = [heading[:5:2] for heading in headingsFinal]
enlacesFinal = [enlace[::2] for enlace in enlaces]

#Visualizo tablas
from google.colab import data_table
def pasarATabla(lista,columnas):
  lista = pd.DataFrame (lista, columns = columnas )
  lista = data_table.DataTable(lista, include_index=True, num_rows_per_page=20)
  display(lista)


pasarATabla(headingsFinal, ['Encabezado', 'Puntuación', 'Agrupados'])
pasarATabla(enlacesFinal, ['Enlaces', 'Puntuación'])

print("-----------------------------------------")
print("*******: ")
print("-----------------------------------------")

#Si quieres visualizar el vocabulario => quita el # de la siguiente línea y dale al play.
#pasarATabla(vocabulario, ['Vocabulario'])

Procesando urls:
https://feriabellezaysalud.com/
https://feriabellezaysalud.com/
https://www.bellezaysalud.tienda/
#
https://www.salud.mapfre.es/cuerpo-y-mente/belleza-y-salud/
https://syb.com.co/
https://masbellezaysalud.info/
https://www.amazon.co.uk/Belleza-y-salud-Health-Beauty/dp/9706660615
http://bellezasalud.es/
https://www.amazon.co.uk/Masajes-Massage-Belleza-Health-Beauty/dp/950768610X
9 páginas escrapeadas correctamente
Incidencias
-----------
timeout error: Invalid URL '#': No scheme supplied. Perhaps you meant https://#? url: #
timeout error: HTTPConnectionPool(host='bellezasalud.es', port=80): Max retries exceeded with url: / (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x7ff12093d5d0>, 'Connection to bellezasalud.es timed out. (connect timeout=10)')) url: http://bellezasalud.es/
Instalando dependencias...


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


2023-10-25 09:44:51.989199: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-25 09:44:51.989267: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-25 09:44:51.989298: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-25 09:44:53.246476: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 59.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
Pasando texto scrapeado a vectores y creando modelo wor

,Encabezado,Puntuación,Agrupados
0,Belleza y Salud,1.0,Belleza y salud
1,Especial Más Belleza y Salud,0.796,
2,Salud Financiera,0.758,
3,Belleza y salud/ Health and Beauty Paperbac...,0.753,
4,Belleza para ti,0.72,
...,...,...,...
144,Fotorejuvenecimiento,-0.032,
145,Lifestyle,-0.027,
146,"Aceite de oliva, proteje tu corazón",-0.024,
147,5 Hábitos que te empobrecen y debes evitar,-0.02,


,Enlaces,Puntuación
0,¿Qué es belleza y salud?,1.0
1,"Belleza y Salud 2023, celebró la pluralidad de...",0.869
2,"La feria Belleza y Salud 2023, presentará el 1...",0.764
3,Sell on Amazon,0.345
4,Amazon Money Store,0.345
...,...,...
109,Payment Methods Help,-0.038
110,Contactos comerciales,-0.024
111,Continue shopping,-0.016
112,Today's Deals,-0.009


-----------------------------------------
*******: 
-----------------------------------------
